# 02 - Modelo Linear Refinado 

Este notebook implementa uma *terceira versão* de um modelo preditivo para estimar a **demanda horária** do pronto-socorro.

O objetivo desta terceira versão é manter a aprimorar e metodologia de divisão de dados de treino e teste,utilizados na versão anterior, mantendo as novas variáveis criados metologia utilizada no modelo anterior. Com isso, começamos a construir uma pipeline mais robusta para implementação do modelo.

Objetivo: aprimorar o modelo linear simples utilizando engenharia de features mais robusta:
- Variáveis categóricas temporais (codificações)
- Melhor validação temporal (TimeSeriesSplit)


Seguiremos avaliando estas etapas:
1. Seleção das features e preparação dos dados
2. Divisão treino/teste respeitando series temporais
3. Treinamento do modelo baseline (Regressão Linear)
4. Predição
5. Avaliação com MAE, RMSE, MAPE e R²
6. Gráficos de diagnóstico

---

# 1. Importar bibliotecas e configurações iniciais

### 1.0 Instalações

In [ ]:
# ! pip install scikit-learn

### 1.1 Importações

In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error, mean_squared_error

import locale

In [ ]:
import sys
sys.path.append("../src")

from features.feature_engineering import create_lag_features, create_rolling_features, add_time_features

### 1.2 Configurações de bibliotecas

In [ ]:
# Desabilitar a limitação de linhas em gráficos do Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

# 2. Importar e tratar dados

### 2.1 Importando dados

In [ ]:
CAMINHO_DADOS = '../data/raw/dataset_pronto_socorro.csv'

df = pd.read_csv(CAMINHO_DADOS)

In [ ]:
df.head()

,datetime,day_of_week,month,is_weekend,temperature,rain_mm,demand
0,2023-01-01 00:00:00,6,1,1,24.483571,0.353269,29.0
1,2023-01-01 01:00:00,6,1,1,21.308678,5.847757,30.0
2,2023-01-01 02:00:00,6,1,1,25.238443,1.141991,30.0
3,2023-01-01 03:00:00,6,1,1,29.615149,0.524987,33.0
4,2023-01-01 04:00:00,6,1,1,20.829233,0.820610,33.0


### 2.2 Tratamento de Dados

In [ ]:
df.drop(columns=['day_of_week'], inplace=True)

In [ ]:
# Converter coluna de data/hora para datetime e definir índice
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

### 2.3 Engenharia de Featurees

In [ ]:
df = df.copy()

# Adicionar variáveis temporais
df = add_time_features(df)

# Lags essenciais: 1 hora, 24h, 48h, 1 semana (168h)
df = create_lag_features(df, lags=[1, 2, 3, 24, 48, 168])

# Rolling windows
df = create_rolling_features(df, windows=[3, 6, 12, 24])

# Remover linhas com NaNs causados pelos lags/rolling
df = df.dropna()

In [ ]:
df.head()

,day_of_week,month,is_weekend,temperature,rain_mm,demand,hour,dayofweek,demand_lag_1,demand_lag_2,...,demand_lag_48,demand_lag_168,demand_roll_mean_3,demand_roll_std_3,demand_roll_mean_6,demand_roll_std_6,demand_roll_mean_12,demand_roll_std_12,demand_roll_mean_24,demand_roll_std_24
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-08 00:00:00,6,1,1,20.773059,2.344874,25.0,0,6,28.0,26.0,...,22.0,29.0,26.333333,1.527525,24.166667,3.060501,24.083333,3.704011,25.583333,3.877658
2023-01-08 01:00:00,6,1,1,18.231319,0.268283,25.0,1,6,25.0,28.0,...,27.0,30.0,26.000000,1.732051,24.833333,2.639444,23.833333,3.511885,25.375000,3.762660
2023-01-08 02:00:00,6,1,1,17.552428,0.192845,22.0,2,6,25.0,25.0,...,24.0,30.0,24.000000,1.732051,24.333333,2.875181,23.583333,3.528026,25.000000,3.623594
2023-01-08 03:00:00,6,1,1,17.920949,1.191458,34.0,3,6,22.0,25.0,...,26.0,33.0,27.000000,6.244998,26.666667,4.082483,24.000000,4.410731,25.250000,4.024382
2023-01-08 04:00:00,6,1,1,21.614491,3.771095,35.0,4,6,34.0,22.0,...,23.0,33.0,30.333333,7.234178,28.166667,5.269409,24.833333,5.441145,25.666667,4.488310


In [ ]:
# Opcional: features sazonais contínuas (útil até pro baseline)

# Sinais "sin" e "cos" transformam hora em um ciclo contínuo, evitando que "23" esteja longe de "0".

# df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
# df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)

# 3 Desenvolvimento de Modelo Preditivo

### 3.1 Seleção das variáveis do modelo

In [ ]:
features = [
    # variáveis temporais
    "hour", "dayofweek", "month", "is_weekend",

    # lags
    "demand_lag_1", "demand_lag_2", "demand_lag_3",
    "demand_lag_24", "demand_lag_48", "demand_lag_168",

    # rolling windows
    "demand_roll_mean_3", "demand_roll_mean_6",
    "demand_roll_mean_12", "demand_roll_mean_24",
    "demand_roll_std_3",  "demand_roll_std_6",
    "demand_roll_std_12", "demand_roll_std_24",
]

target = 'demand'


In [ ]:
X = df[features]
y = df[target]

### 3.2 Divisão dos dados em treino/teste

In [ ]:
# Dividindo os dados em treino e teste, a partir da função TimseSeriesSplit, que divide os dados 
tscv = TimeSeriesSplit(n_splits=10)

### 3.3 Validação Regressão Linear

Ao trabalhar com séries temporais, a ordem dos dados é fundamental e não deve ser embaralhada. Por isso, utilizamos a classe **TimeSeriesSplit**, que implementa um esquema de validação cruzada adequado para esse tipo de dado. Em vez de realizar divisões aleatórias, o TimeSeriesSplit cria múltiplos splits incrementais, sempre respeitando a sequência temporal.

Em cada iteração, um bloco inicial de dados é usado para treino e, logo em seguida, um bloco subsequente é reservado para teste. O conjunto de treino sempre cresce ao longo das divisões, simulando um cenário real de previsão em que apenas informações do passado podem ser usadas para prever o futuro.

**Como avaliar a abordagem:**

O TimeSeriesSplit permite verificar a consistência do modelo em diferentes janelas temporais, mostrando se ele generaliza bem ao longo do tempo. Se o desempenho for estável entre as diversas iterações, isso indica que o modelo está capturando padrões robustos. Caso haja grande variação entre os splits, pode ser sinal de sazonalidade não modelada, mudanças estruturais na série ou necessidade de features mais fortes.

In [ ]:
# Variável para armazenar os resultados
results = []

for split_i, (train_index, test_index) in enumerate(tscv.split(df), start=1):
    train = df.iloc[train_index]
    test  = df.iloc[test_index]

    X_train = train[features]
    y_train = train[target]
    X_test  = test[features]
    y_test  = test[target]

    model = LinearRegression()
    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    # Métricas
    mae  = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mape = np.mean(np.abs((y_test - preds) / y_test)) * 100
    r2   = r2_score(y_test, preds)

    # Armazenar no resultado
    results.append({
        "split": split_i,
        "train_start": train.index.min(),
        "train_end":   train.index.max(),
        "test_start":  test.index.min(),
        "test_end":    test.index.max(),
        "MAE": mae,
        "RMSE": rmse,
        "MAPE (%)": mape,
        "R2": r2
    })

# DataFrame final
df_results = pd.DataFrame(results)
df_results

,split,train_start,train_end,test_start,test_end,MAE,RMSE,MAPE (%),R2
0,1,2023-01-08,2023-03-14 22:00:00,2023-03-14 23:00:00,2023-05-19 15:00:00,8.870520e-15,1.143424e-14,3.735417e-14,1.0
1,2,2023-01-08,2023-05-19 15:00:00,2023-05-19 16:00:00,2023-07-24 08:00:00,9.998062e-15,1.229818e-14,5.094273e-14,1.0
2,3,2023-01-08,2023-07-24 08:00:00,2023-07-24 09:00:00,2023-09-28 01:00:00,6.145722e-15,7.935163e-15,3.569038e-14,1.0
3,4,2023-01-08,2023-09-28 01:00:00,2023-09-28 02:00:00,2023-12-02 18:00:00,2.784273e-14,2.929233e-14,1.673600e-13,1.0
4,5,2023-01-08,2023-12-02 18:00:00,2023-12-02 19:00:00,2024-02-06 11:00:00,1.375466e-14,1.728105e-14,6.311897e-14,1.0
5,6,2023-01-08,2024-02-06 11:00:00,2024-02-06 12:00:00,2024-04-12 04:00:00,1.071671e-14,1.352315e-14,4.404501e-14,1.0
6,7,2023-01-08,2024-04-12 04:00:00,2024-04-12 05:00:00,2024-06-16 21:00:00,1.754279e-14,2.066503e-14,7.810205e-14,1.0
7,8,2023-01-08,2024-06-16 21:00:00,2024-06-16 22:00:00,2024-08-21 14:00:00,1.024982e-14,1.308498e-14,5.221299e-14,1.0
8,9,2023-01-08,2024-08-21 14:00:00,2024-08-21 15:00:00,2024-10-26 07:00:00,1.117460e-14,1.328195e-14,6.564592e-14,1.0
9,10,2023-01-08,2024-10-26 07:00:00,2024-10-26 08:00:00,2024-12-31 00:00:00,1.741607e-14,1.941352e-14,9.471975e-14,1.0


### 3.4 Treino do modelo final

In [ ]:
model_final = LinearRegression()
model_final.fit(X,y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


# 4. Análise detalhada do resultado

### 4.1 Criar dataframe com resultados

In [ ]:
df_eval = pd.DataFrame({
    "y_true": y_test,
    "y_pred": preds
})

In [ ]:
# Erro residual
# Residual positivo = modelo subestimou
# Residual negativo = modelo superestimou

df_eval["residual"] = df_eval["y_true"] - df_eval["y_pred"]

In [ ]:
df_eval.head()

,y_true,y_pred,residual
datetime,,,
2024-10-26 08:00:00,22.0,22.0,1.776357e-14
2024-10-26 09:00:00,20.0,20.0,1.776357e-14
2024-10-26 10:00:00,21.0,21.0,2.486900e-14
2024-10-26 11:00:00,22.0,22.0,2.486900e-14
2024-10-26 12:00:00,19.0,19.0,2.131628e-14


## 4.2 Métricas de Avaliação

Neste cenário, em que estamos utilizando validação cruzada, esta avaliação é feita apenas para a última janela de dados usada.

In [ ]:
mse = mean_squared_error(df_eval["y_true"], df_eval["y_pred"])
print(f"MAE : {mae:.2f}")

MAE : 0.00


In [ ]:
# RMSE penaliza erros grandes → ótimo para detectar picos que o modelo perdeu.
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.2f}")

RMSE: 0.00


In [ ]:
# MAPE ajuda a entender o erro percentual, mas pode distorcer quando valores são pequenos.
mape = np.mean(np.abs(df_eval["residual"] / df_eval["y_true"])) * 100
print(f"MAPE: {mape:.2f}%")

MAPE: 0.00%


## 4.3 Distribuição dos Resíduos

In [ ]:
# Se os resíduos não forem centrados em 0 → viés no modelo.
# Caudas pesadas → eventos extremos não capturados.

In [ ]:
# Histograma
hist =alt.Chart(df_eval.reset_index()).mark_bar().encode(
    x=alt.X("residual", bin=alt.Bin(maxbins=50), title='Distribuição dos Resíduos'),
    y=alt.Y('count()',title='Frequência')
)
# Linha vertical no zero
linha_zero = (
    alt.Chart(pd.DataFrame({"x": [0]}))
    .mark_rule(color="black")
    .encode(x="x:Q")
)
# KDE (curva suavizada) — opcional
kde = (
    alt.Chart(df_eval.reset_index())
    .transform_density(
        "residual",
        as_=["residual", "density"]
    )
    .mark_line(color="red")
    .encode(
        x="residual:Q",
        y="density:Q"
    )
)

(hist + linha_zero + kde).resolve_scale(
    y="independent"
).properties(
    width=600,
    height=300,
    title="Distribuição dos resíduos"
)


alt.LayerChart(...)

### 4.4 Error por Hora do dia

- Este gráfico revela se o modelo falha no começo, meio ou final do dia.
- Se os erros forem sistematicamente altos em certas horas → falta de features temporais.

In [ ]:
df_hour = df_eval.copy()
df_hour["hour"] = X_test["hour"]

hour_mae = df_hour.groupby("hour",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
hour_mae.head()

,hour,residual
0,0,2.110097e-14
1,1,1.956725e-14
2,2,1.664310e-14
3,3,1.582324e-14
4,4,1.634248e-14


In [ ]:
chart = (
    alt.Chart(hour_mae)
    .mark_bar()
    .encode(
        x=alt.X("hour:O", title="Hora do dia"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por hora do dia"
    )
)

chart

alt.Chart(...)

### 4.5 Erro por dia da Semana

- A demanda hospitalar costuma variar muito entre domingo e segunda.
- Se o baseline não captura → será necessário adicionar lags.

In [ ]:
df_dow = df_eval.copy()
df_dow["dayofweek"] = X_test["dayofweek"]

dow_mae = df_dow.groupby("dayofweek",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
dow_mae.head()

,dayofweek,residual
0,0,1.677917e-14
1,1,1.796412e-14
2,2,1.944946e-14
3,3,1.777179e-14
4,4,1.563358e-14


In [ ]:
chart = (
    alt.Chart(dow_mae)
    .mark_bar()
    .encode(
        x=alt.X("dayofweek:O", title="Dia da Semana"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por dia da semana"
    )
)

chart

alt.Chart(...)

### 4.6 Comparação Real vs. Predição agregado por dia

- Linear Regression pode acertar nível médio mas errar amplitude.
- Se o modelo suaviza demais → pode exigir modelos não lineares.

In [ ]:
df_daily = df_eval[["y_true", "y_pred"]].resample("D").sum().reset_index()
df_daily.head()

,datetime,y_true,y_pred
0,2024-10-26,313.0,313.0
1,2024-10-27,515.0,515.0
2,2024-10-28,397.0,397.0
3,2024-10-29,398.0,398.0
4,2024-10-30,378.0,378.0


In [ ]:
# Converte para formato long (necessário para múltiplas linhas no Altair)
df_long = df_daily.melt(id_vars="datetime", value_vars=["y_true", "y_pred"],
                       var_name="tipo", value_name="valor")
df_long.head()

,datetime,tipo,valor
0,2024-10-26,y_true,313.0
1,2024-10-27,y_true,515.0
2,2024-10-28,y_true,397.0
3,2024-10-29,y_true,398.0
4,2024-10-30,y_true,378.0


In [ ]:
chart = (
    alt.Chart(df_long)
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("valor:Q", title="Demanda"),
        color=alt.Color("tipo:N", title="Série", scale=alt.Scale(
            domain=["y_true", "y_pred"],
            range=["black", "steelblue"]
        )),
        tooltip=["data:T", "tipo:N", "valor:Q"]
    )
    .properties(
        width=1200,
        height=500,
        title="Demanda diária — Real vs Prevista"
    )
)

chart

alt.Chart(...)

### 4.7 Resíduos ao longo de tempo

- Ver se há períodos em que o modelo erra sistematicamente (viés temporal).
- Ver se há heterocedasticidade (erro aumenta em períodos de pico).

In [ ]:
# Linha dos resíduos
residual_line = (
    alt.Chart(df_eval.reset_index())
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("residual:Q", title="Resíduo")
    )
)

# Linha horizontal em zero
linha_zero = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule(color="black")
    .encode(y="y:Q")
)

chart = (
    (residual_line + linha_zero)
    .properties(
        width=1200,
        height=400,
        title="Resíduos ao longo do tempo"
    )
)

chart

alt.LayerChart(...)

### 4.8 Interpretação dos coeficientes do modelo

- coef > 0 → aumenta demanda
- coef < 0 → reduz demanda
  
- Permite validar coerência com conhecimento hospitalar.

In [ ]:
coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_
})

coef_df.sort_values("coef", ascending=False)

,feature,coef
10,demand_roll_mean_3,3.000000e+00
1,dayofweek,1.887379e-15
2,month,1.432720e-15
13,demand_roll_mean_24,1.230612e-15
8,demand_lag_48,4.669304e-16
11,demand_roll_mean_6,4.467609e-16
0,hour,3.459164e-16
6,demand_lag_3,3.297217e-16
7,demand_lag_24,1.402068e-16
14,demand_roll_std_3,5.157915e-17


### 4.9 Importância padronizada (coef*std)

- Isso mostra quais variáveis mais impactam a previsão na prática.
- Muito útil para justificar a evolução do modelo.

In [ ]:
stds = X_train.std()

coef_imp_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_,
    "std": stds,
})

coef_imp_df["importance_std"] = coef_imp_df["coef"] * coef_imp_df["std"]
coef_imp_df.sort_values("importance_std", ascending=False)

,feature,coef,std,importance_std
demand_roll_mean_3,demand_roll_mean_3,3.000000e+00,4.586986,1.376096e+01
month,month,1.432720e-15,3.206848,4.594517e-15
demand_roll_mean_24,demand_roll_mean_24,1.230612e-15,3.605755,4.437285e-15
dayofweek,dayofweek,1.887379e-15,2.000062,3.774876e-15
demand_lag_48,demand_lag_48,4.669304e-16,5.225507,2.439948e-15
hour,hour,3.459164e-16,6.923186,2.394844e-15
demand_roll_mean_6,demand_roll_mean_6,4.467609e-16,4.326538,1.932928e-15
demand_lag_3,demand_lag_3,3.297217e-16,5.232776,1.725360e-15
demand_lag_24,demand_lag_24,1.402068e-16,5.231200,7.334496e-16
demand_roll_std_3,demand_roll_std_3,5.157915e-17,1.432059,7.386441e-17


# 5. Conclusão

- Conforme esperado, mantivemos o ótimo resultado do modelo para o conjunto de dados observado, tendo agora uma validação cruzada mais robusta para garantir que o bom desempenho do modelo ocorre em diversas janelas de tempo.

Próximos passos:
- Testar outros modelos de regressão